# Basic First Model to Set a B

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics
import pipeline_tools as pt
import xgboost as xgb

# Load Train Data

In [2]:
train_dir = '../tabular-playground-series-mar-2021/train.csv'

df = pd.read_csv(train_dir)


In [3]:
cat_features = [f'cat{i}' for i in range(19)]
num_features = [f'cont{i}' for i in range(11)]
target = 'target'

# Build and Fit the pipe

In [4]:
X = df[cat_features + num_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [5]:
pre_pipe = pt.standard_preprocessing_pipe(num_features=num_features, cat_features=cat_features)

pipe = Pipeline([
    ('pre_pipe', pre_pipe),
    ('learn', xgb.XGBClassifier())
])

In [ ]:
pipe.fit(X_train, y_train)

/Users/stuartday/opt/anaconda3/envs/kaggle_march_tab/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:59:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Score our model

In [ ]:
test_prediction = pipe.predict_proba(X_test)[:,1]

predictions = pipe.predict(X_test)

score = metrics.roc_auc_score(y_test, test_prediction)
accuracy = metrics.accuracy_score(y_test, predictions)

print(f'Area under ROC of Model On Test Set - {score:,.2%}')
print(f'Accuracy - {accuracy:,.2%}')

# Prepare Submission

In [ ]:
test_dir = '../tabular-playground-series-mar-2021/test.csv'


df_test = pd.read_csv(test_dir)

In [ ]:
df_test['target'] = pipe.predict(df_test[cat_features + num_features])

In [ ]:
# df_test[['id', 'target']].to_csv('submission.csv', index=False)